# 1 Формализация постановки задачи

а) Если не будет размечанных данных - то это задача кластеризации. Если в итоге будут размеченные данные, то задача классификации. Удобнее, конечно, с размеченными данными, так как в кластеризации надо будет еще контролировать количество кластеров + понимать, нужные ли кластеры вышли.
Регрессию тоже можно пытаться применять (но тут она скорее как бы вместо классификации...)

b) Целевые значения - метки, хороший отзыв или плохой, так же можно использовать большее количество меток (хороший, плохой, нейтральный), или еще больше меток (это уже зависит от того, какие данные у нас появятся, если данные с метками 1-5, то такие метки можно и оставить) (или вероятность "хорошести" отзыва [0, 1]).
Скорее всего: хороший, плохой, нейтральный (когда мы точно не уверена)

Как показываться в демострации?
Классы в духе, 0, 1 - не очень, так как неинтуитивно, можно явно текстом прописать. Но для визуализации лучше всего, наверное, выделять цветом. Типа зеленые - хорошие, классые - плохие. (хорошие == позитивные, плохие == негативные)
Так же есть способ предоставлять окно для ввода фидбека, которое по нажатию кнопки будет выдавать результат.

с) Если показывать цифру и говорить accuracy 98%, recall 0.53 - это может быть непонятно тому, кому мы показываем.
Можно явно просто словами писать: 98 правильно помеченных отзывов среди 100.
Или же визуализировать какими-нибудь диаграммами, например круговая диаграмма должна неплохо подойти (зависит так же от количества классов, если их много, то выйдет перегруженно).

d) Это стоит спросить у заказчика, наверное.
А вообще в идеале - на всех. В идеале должна получиться модель, которая не чувствительна к "пользовательским ошибкам", люди, которые пишут отзывы же не знают об этих ограничениях.

Возможно, есть смысл накладывать ограничения на язык (напр русский, если банк русский и его аудитория русская), это облегчит обучение модели и не сильно повредит. А опечатки и сленг встречаются, скорее всего, часто. Но это надо наверное напрямую посмотреть на отзывы и понять, есть ли в них эти "минусы". 

# 2 Сбор данных

Со скрапи у меня были проблемы, так что быстрее было по хардкоду.

In [127]:
import requests
import bs4
from tqdm import tqdm

In [128]:
X = []
y = []
for item in tqdm(range(10100000, 10130000)):
    response = requests.get('http://www.banki.ru/services/responses/bank/response/{item}/'.format(item=item))
    if response.status_code != 200:
        continue
    html_doc = response.text
    soup = bs4.BeautifulSoup(html_doc, 'html.parser')
    feedback_rating_tag = soup.find_all(name="meta", attrs={"itemprop": "value"})
    if len(feedback_rating_tag) != 0:
        feedback_rating = feedback_rating_tag[0].get_attribute_list("content")[0]
        feedback_text = soup.find_all(name="div", attrs={"itemprop": "description"})[0].get_text()
        X.append(feedback_text)
        y.append(int(feedback_rating))

100%|██████████| 30000/30000 [2:47:40<00:00,  2.98it/s]  


In [129]:
print(len(X))
print(len(y))

11565
11565


Так как в собранных данных рейтинг дается от 1 до 5, то так и оставим.
Можно было бы, конечно, и уменьшить колисевтво классов.
Так бы мы могли и расширяться на новые данные с других сайтов (не banki.ru).

# 3 Модель

In [130]:
from sklearn.feature_extraction.text import CountVectorizer

In [131]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(X)
dataset = count_vectorizer.transform(X)

In [132]:
print(dataset.shape)

(11565, 86297)


In [133]:
from sklearn.ensemble import GradientBoostingClassifier

In [134]:
clf = GradientBoostingClassifier()

In [135]:
from sklearn.model_selection import cross_val_score

In [136]:
from sklearn.metrics import make_scorer

В качестве скора просто возьмем accyracy (отношение правильно угаданных ко всем).

In [137]:
def acc(y, y_pred):
    matched = 0
    non_matching_sum = 0
    for i in range(len(y)):
        if y[i] == y_pred[i]:
            matched += 1
            non_matching_sum += y[i] - y_pred[i]
    return matched / len(y)

scorer = make_scorer(acc)

In [138]:
cross_val_score(clf, dataset, y, scoring=scorer)

array([ 0.76607884,  0.77250324,  0.76647639])

In [139]:
clf.fit(dataset, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [140]:
y_pred = clf.predict(dataset)

In [144]:
print(acc(y, y_pred))

0.7961954172070903


Неплохое качество.(в том числе на кросс валидации). Рандом - это точность 0.20. 

Сохраним модель.

In [145]:
import pickle

In [146]:
with open('model', 'wb') as model_file:
    pickle.dump(clf, model_file)

In [147]:
with open('vectorizer', 'wb') as vectorizer_file:
    pickle.dump(count_vectorizer, vectorizer_file)

In [149]:
with open('vectorizer', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

# Выполнение

На выполнение и правда понадобилось несколько часов (у меня лично часа 4 заняло). Спасибо!